# Notebook para revisión de dataframes y resultados.
# -Pruebas internas-

In [72]:
import re
import sys
import os, glob
import pandas as pd 
import numpy as np
## Libreria que se integra con Teradata
import giraffez

## Lectura de archivos

In [174]:
df_bteq    = pd.read_csv("LAC_Levantamiento_Bteq.csv",  sep='|' ) 
df_vistas  = pd.read_csv("LAC_Levantamiento_Vistas.csv",  sep='|') 
df_sp      = pd.read_csv("LAC_Levantamiento_StoredProcedures.csv" ,  sep='|') 

In [27]:
df_cim     = pd.read_csv("LAC_Levantamiento_CIM.csv" ,  sep='|') 

## Revisión de esquemas y tablas

In [175]:
columns_input = ['ESQUEMA_INPUT','TABLA_INPUT']
columns_output = ['ESQUEMA_OUTPUT','TABLA_OUTPUT']

columns = ['ESQUEMA_OUTPUT','TABLA_OUTPUT','ESQUEMA_INPUT','TABLA_INPUT']

In [176]:
df_bteq_groupby_tables = df_bteq[columns].groupby(columns).last().reset_index() 
df_sp_groupby_tables  = df_sp[columns].groupby(columns).last().reset_index() 
df_vistas_groupby_tables = df_vistas[columns].groupby(columns).last().reset_index() 

df_bteq_groupby_tables.to_csv("Resultados/resultados_tablas_bteq.csv", header=True, index=None, sep='|', mode='w')
df_sp_groupby_tables.to_csv("Resultados/resultados_tablas_sp.csv", header=True, index=None, sep='|', mode='w')
df_vistas_groupby_tables.to_csv("Resultados/resultados_tablas_vistas.csv", header=True, index=None, sep='|', mode='w')

In [30]:
df_cim_groupby_tables = df_cim[columns_input].groupby(columns_input).last().reset_index() 

df_cim_groupby_tables.to_csv("Resultados/resultados_tablas_cim.csv", header=True, index=None, sep='|', mode='w')

### _Revisión por casos específicos_

In [172]:
#filter1 = df_bteq['ESQUEMA_OUTPUT'] == 'MNA_SIMUCON'  
filter1 = df_bteq['ESQUEMA_INPUT'] == 'X' 

In [173]:
pd.set_option('display.max_colwidth',-1)
df_bteq_tmp = df_bteq[filter1]
df_bteq_tmp['ARCHIVO']

116579    ./ArchivosProcesar/Input_Analitico/Personas/20_Campannas/217_Masivo_Masivo_Tablon_Analitico_BCI/BTEQs/MP_01_Creacion_Publicos_Objetivos.sql
Name: ARCHIVO, dtype: object

In [160]:
filter2 = df_sp['ESQUEMA_OUTPUT'] == 'CA_COMPONENT_RUN'
df_sp_tmp = df_sp[filter2]
df_sp_tmp['ARCHIVO']

Series([], Name: ARCHIVO, dtype: object)

## Revisión de campos

In [22]:
campos_reglas = ['COLUMNA','VARIABLE','LOGICA_NEGOCIO']

In [23]:
df_bteq_groupby_columns = df_bteq[campos_reglas].groupby(campos_reglas).last().reset_index() 
df_sp_groupby_columns  = df_sp[campos_reglas].groupby(campos_reglas).last().reset_index() 
df_vistas_groupby_columns = df_vistas[campos_reglas].groupby(campos_reglas).last().reset_index() 

df_bteq_groupby_columns.to_csv("Resultados/resultados_campos_reglas_bteq.csv", header=True, index=None, sep='|', mode='w')
df_sp_groupby_columns.to_csv("Resultados/resultados_campos_reglas_sp.csv", header=True, index=None, sep='|', mode='w')
df_vistas_groupby_columns.to_csv("Resultados/resultados_campos_reglas_vistas.csv", header=True, index=None, sep='|', mode='w')

In [32]:
df_cim_groupby_columns = df_cim[campos_reglas].groupby(campos_reglas).last().reset_index() 

df_cim_groupby_columns.to_csv("Resultados/resultados_campos_reglas_cim.csv", header=True, index=None, sep='|', mode='w')

## Union de tablas de los archivos

In [168]:
df_tablas_bteq    = pd.read_csv("Resultados/resultados_tablas_bteq.csv",  sep='|' ) 
df_tablas_vistas  = pd.read_csv("Resultados/resultados_tablas_sp.csv",  sep='|') 
df_tablas_sp      = pd.read_csv("Resultados/resultados_tablas_vistas.csv" ,  sep='|') 

In [60]:
concat_df = pd.concat([df_tablas_bteq, df_tablas_vistas, df_tablas_sp])

In [36]:
concat_df.to_csv("Resultados/union_tablas_archivos.csv", header=True, index=None, sep='|', mode='w')

## Busca fecha de creación de todas las tablas de la unión.

In [63]:
## Insertar en una sola columna las tablas output e input

df_1 = concat_df[['ESQUEMA_OUTPUT','TABLA_OUTPUT']].drop_duplicates()   
df_2 = concat_df[['ESQUEMA_INPUT','TABLA_INPUT']].drop_duplicates()   

df1 = df_1.rename(columns={'ESQUEMA_OUTPUT': 'ESQUEMA', 'TABLA_OUTPUT': 'TABLA'})
df2 = df_2.rename(columns={'ESQUEMA_INPUT': 'ESQUEMA', 'TABLA_INPUT': 'TABLA'})

concat_df_2 = pd.concat([df1, df2])
concat_df_2 = concat_df_2.drop_duplicates()  


In [37]:
## Configuracion de conexion
td_config = {
    "username": "exajibl",
    "password": "xips2591",
    "host": "dataware.bci.cl"}

In [131]:
def get_creationdates_tables(schemas, tables):

    query_show_date_creates = """SELECT DATABASENAME, TABLENAME, CREATETIMESTAMP, LASTACCESSTIMESTAMP
                                   FROM DBC.TABLES 
                                  WHERE DATABASENAME IN {} 
                                    AND TABLENAME IN {} """.format(schemas, tables)

    #print(query_show_date_creates)

    with giraffez.BulkExport(query_show_date_creates, **td_config) as export:
        dataset = export.to_list()
        
    headers = ['DATABASENAME','TABLENAME', 'CREATETIMESTAMP','LASTACCESSTIMESTAMP']

    df = pd.DataFrame(dataset, columns = headers)
    
    return df

In [98]:
def unique(_list):
    """
    Makes the list have unique items only and maintains the order
    list(set()) won't provide that
    :type _list list
    :rtype: list
    """
    ret = []

    for item in _list:
        if item not in ret:
            ret.append(item)

    return ret

In [132]:
list_all_tables = concat_df_2.values.tolist()

def convert_to_tuple(list): 
    return (*list,)

# Retorna una tupla de todos los esquemas
list_schemas = [i[0] for i in list_all_tables]
list_schemas = unique(list_schemas)
tuple_schemas = convert_to_tuple(list_schemas)


# Retorna una tupla de todas las tablas
list_tables = [i[1] for i in list_all_tables]
list_tables = unique(list_tables)
tuple_tables = convert_to_tuple(list_tables)


In [ ]:
# Lo convierte a tupla para hacer un 'IN'
df_total_creationdate = get_creationdates_tables(tuple_schemas, tuple_tables)

#df_total_creationdate.to_csv("Resultados/fechacreacion_tablas_teradata.csv", header=True, index=None, sep='|', mode='w')


## Revision de tablas en las vistas

In [192]:
df_t_v          = pd.read_csv("Resultados/resultados_logs_airflow.csv" ,  sep='|') 
df_t_v_group_by = df_t_v.VISTA_OUTPUT.unique()

for item in df_t_v_group_by:
    print(item)

ARMVIEWS.RMA_PRUEBA_PERFILAR
ARMVIEWS.RMA_DECIL_PROP_SEG_MP
ARMVIEWS.RMS_LKG_EVET_SEGM_SUC_A
ARMVIEWS.D_CAMPANAS_BATCH_SUC_EVEREST_A
ARMVIEWS.RMS_CUST_COLLATERAL_TCR
ARMVIEWS.RMU_CRM_PYME_MARGEN_DISP_A
ARMVIEWS.RMS_LKG_EVT_ATC_SUC_V3_170504
ARMVIEWS.RMA_PARTY_CARD_ATTR_D
ARMVIEWS.RMU_LKG_EVT_CRM_D_1180726
ARMVIEWS.RMA_SOCIOS_A
ARMVIEWS.RMA_SEGMENTO_INR_A
ARMVIEWS.RMS_CUST_CON_TELE_A
ARMVIEWS.RMJ_PROS_ACCOUNT_20180913
ARMVIEWS.RMS_LKG_EVT_CRM_AUDITORIA
ARMVIEWS.RMS_INVDAP_OFERTA_A_ER
ARMVIEWS.RMU_EJEC_FFVV_CP_PLN_1170705
ARMVIEWS.RMA_NBA_HIP_OFER
ARMVIEWS.RMJ_PROD_GROUP_CATEGORY
ARMVIEWS.CRM_SOAP_RNVM_7ANOS
ARMVIEWS.D_RCI_CURRENT_COMM_TRACKING
ARMVIEWS.RMA_EVT_CRM_NOPRIO_INT_RISGO_A
ARMVIEWS.RMS_NCLI_COMPRA_TEL_A
ARMVIEWS.RMU_FONOS_PLN_A
ARMVIEWS.REP_NOMINAS_ATM_CON
ARMVIEWS.RMA_PYME_CELULA_RNV_M
ARMVIEWS.RMS_CRM_MAX_ID_PRD_CHAN_A
ARMVIEWS.CGTD_RMA_TABLON_CAMPANA_A
ARMVIEWS.RMS_LKG_EVT_INVP_WEB_A
ARMVIEWS.RMS_COMEX_SEGTO_NOMINA_1180525
ARMVIEWS.RMS_LKG_EVET_LSG_SUC_A
ARMVIEWS.RMA_PBD_CO